# Copyright

<PRE>
Jelen iTorch notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott "Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült. 
A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A notebook bármely részének újra felhasználása, publikálása csak a szerző írásos beleegyezése esetén megegengedett.

2016 (c) Tóth Bálint Pál (toth.b kukac tmit pont bme pont hu)
</PRE>

# Bevezető
Ebben a notebookban megvizsgáljuk a Torch7 alapú implementációt a XOR probléma megoldására CUDA alapon. Első lépésként betöltjük a torch és nn (Neural Network) és ezek CUDA-s változatát töltjük be (cutorch, cunn). Ezen túl még használni fogjuk az optim csomagot.

In [1]:
require "torch"
require "nn"
require "cutorch"
require "cunn"
require "optim"

Következő lépésként létrehozzuk a neurális hálózatot:

In [2]:
inputs = 2; 
hiddens = 20;
outputs = 1;

model = nn.Sequential();  
model:add(nn.Linear(inputs, hiddens))
model:add(nn.Tanh())
model:add(nn.Linear(hiddens, outputs))

Ezután megadjuk a költségfüvvényt (négyzetes hiba):

In [3]:
criterion = nn.MSECriterion()


Majd definiáljuk az adatokat (első két oszlop a két bemenet, utolsó oszlop a kimenet):

In [4]:
data=torch.Tensor(4,3)
data[1]=torch.Tensor({0,0,0})
data[2]=torch.Tensor({0,1,1})
data[3]=torch.Tensor({1,0,1})
data[4]=torch.Tensor({1,1,0})
print(data)

 0  0  0
 0  1  1
 1  0  1
 1  1  0
[torch.DoubleTensor of size 4x3]



Következő lépésként tegyünk át mindent a GPU-ra és nézzük meg a hálózatunkat:

In [5]:
model:cuda()
criterion:cuda()
data=data:cuda()

print(model)

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> output]
  (1): nn.Linear(2 -> 20)
  (2): nn.Tanh
  (3): nn.Linear(20 -> 1)
}
{
  gradInput : CudaTensor - empty
  modules : 
    {
      1 : 
        nn.Linear(2 -> 20)
        {
          gradBias : CudaTensor - size: 20
          weight : CudaTensor - size: 20x2


          _type : torch.CudaTensor
          output : CudaTensor - empty
          gradInput : CudaTensor - empty
          bias : CudaTensor - size: 20
          gradWeight : CudaTensor - size: 20x2
        }
      2 : 
        nn.Tanh
        {
          gradInput : CudaTensor - empty
          _type : torch.CudaTensor
          output : CudaTensor - empty
        }
      3 : 
        nn.Linear(20 -> 1)
        {
          gradBias : CudaTensor - size: 1
          weight : CudaTensor - size: 1x20
          _type : torch.CudaTensor
          output : CudaTensor - empty
    

      gradInput : CudaTensor - empty
          bias : CudaTensor - size: 1
          gradWeight : CudaTensor - size: 1x20
        }
    }
  _type : torch.CudaTensor
  output : CudaTensor - empty
}


Fentebb megfigyelhetjük, hogy a hálózatnak van egy output és egy gradInput belső változója. 

Ezután indulhat a tanítás:

In [6]:
local coefL1 = 0--.001 -- L1 regularizáció együtthatója
local coefL2 = 0.001 -- L2 regularizáció együtthatója

-- az SGD algoritmus beállításai
local optimState = {
      learningRate = 0.05,
      weightDecay = 0,
      momentum = 0,
      nesterov = false,
      dampering = 0,
      learningRateDecay = 1e-5
    }
 
-- a súlyok és a gradiensek lekérdezése, 1-1 view-t hoz létre
params, gradParams = model:getParameters()
epochs = 400 -- azért nem local, mert még később használjuk

-- megvalósítjuk a mini-batch tanítást (bár XOR esetén nem sok értelme van, de így tudjátok másra is haszhnálni!)
local batchSize=2
-- nehogy az adatnál nagyobb mini-batch méretet adjunk meg
if (batchSize>data:size(1)) then error() end 
-- itt a második dimenziót mindig az adatoknak megfelelően kell állítani
local batchInputs = torch.Tensor(batchSize, data:size(2)-1):cuda()
-- ha több kimenet van, akkor itt is
local batchLabels = torch.DoubleTensor(batchSize):cuda()

losses = torch.Tensor(epochs) -- azért nem local, mert még később használjuk

print("Training started with batch size: " .. batchSize)
for i = 1,epochs do
    -- véletlenszerű sorrendben fogjuk beolvasni az adatokat, ehhez egy random permutációt készítünk
    local shuffle = torch.randperm(data:size(1))

    for k=1,data:size(1),batchSize do
        no=1
        for j=k,math.min(k+batchSize-1,data:size(1)) do
          batchInputs[no] = data[{shuffle[j],{1,2}}]
          batchLabels[no] = data[{shuffle[j],3}]
          no=no+1
        end
        
        -- ezt a lokális függvényt adjuk majd az optim-nak (optimizációs algoritmus)
        -- bemenete a súlyok (params)
        -- kimenete a költségfüggvény szerinti hiba és a súlyokhoz kiszámolt gradiensek 

        local function feval(params)
            gradParams:zero() -- korábbi gradiens kinullázása

            -- kiszámoljuk a háló kimenetét, egyúttal a háló belső "output" változóját frissíti
            local outputs = model:forward(batchInputs) 
            -- kiszámoljuk a költségfüggvény szerinti hibát
            local loss = criterion:forward(outputs, batchLabels) 
            -- kiszámoljuk a gradienseket (@loss/@W) a kimenet függvényében (deltával jelöltük az előadásban)
            local dloss_doutput = criterion:backward(outputs, batchLabels)
            -- kiszámoljuk a gradienst a bemenet függvényében és frissítjük a gradInput belső változót
            -- előadáson ez az a lépés, amikor a "delta"-t összeszorztuk az adott réteg kimenetével
            model:backward(batchInputs, dloss_doutput)

            -- az L1 és L2 regularizáció megvalósítása
            if coefL1 ~= 0 or coefL2 ~= 0 then
                    local norm,sign= torch.norm,torch.sign

                    -- a költségfüggvény kibővítése az L1 és L2 regularizációval
                    loss = loss + coefL1 * norm(params,1)
                    loss = loss + coefL2 * norm(params,2)^2/2

                    -- gradiensek kibővítése
                    gradParams:add( sign(params):mul(coefL1) + params:clone():mul(coefL2) )
            end

            -- a gradParams a gradInput belső változónak egy view-ja, ezt adjuk vissza
            return loss,gradParams
        end
        _,loss = optim.sgd(feval, params, optimState)
        losses[i]=loss[1]
    end
end
print("Training done.")

Training started with batch size: 2	


Training done.	


Teszteljük a betanított hálót, hogy mit generál az egyes bemenetekre:

In [7]:
print(model:forward(data[{{},{1,2}}]))
print(data[{{},3}])

 0.0047
 0.9954
 0.9956
 0.0072
[torch.CudaTensor of size 4x1]



 0
 1
 1
 0
[torch.CudaTensor of size 4]



Végül jelenítsük meg a hiba csökkenését:

In [8]:
Plot = require "itorch.Plot"
 
local x = torch.linspace(0,epochs,epochs)
local plot = Plot()
plot:line(x, losses,'red' ,'Training loss')
plot:legend(true):title('Training XOR with simple model')
plot:draw()

Végül pedig kiiratjuk a súlyok értékét. Emlékeztető: L1 regularizáció ritka súly mátrixhoz vezet, az L2 regularizáció pedig eltünteti a nagy értékeket a súlymátrixból és egységesen kis értékek (de nem ~0) fog szerepelni:

In [9]:
print(params)

 0.4409
-0.7972
 0.3999
 0.2952
 0.1134
-0.5660
-0.6244
 0.2559
-0.0983
 0.2672
 0.5093
 0.4561
-0.4071
-0.2010
 0.1474
-0.7033
 0.6499
 0.2482
-0.8339
-0.8090
 0.2060
-0.0205
-0.6146
-0.4691
-0.4581
-0.4097
 0.5001
 0.7145
 0.2499
-0.5042
-1.1107
-1.1399
-0.9272
 0.4913
-1.1277
-1.1041
 0.2671
 0.5485
-0.8074
 0.0411
-0.3765
 0.2815
 0.3542
-0.6036
-0.3884
-0.6844
 0.1645
-0.2504
-0.5663
-0.0175
 0.1722
-0.3942
-0.4248
-0.8868
-0.4266
 0.1506
-0.3560
 0.0773
 0.5342
 0.3515
 0.6296
 0.0735
 0.1050
 0.3377
 0.0998
-0.4631
 0.2654
 0.3966
-0.2788
-0.4786
-0.2423
-0.1820
 0.0019
-0.7088
 0.4306
-0.8177
 0.6322
-0.9340
 0.0199
 0.2645
-0.0229
[torch.CudaTensor of size 81]



Próbáljátok ki ugyanezt L1 regularizációval (ekkor állítsátok fel az epoch számát pl. 15000-ra)!